In [2]:
import pystac
from pystac import Link, Asset
from datetime import datetime

# ML AOI STAC Extension

In [3]:
from pystac.extensions.label import LabelExtension
from pystac.extensions.label import LabelType
from pystac.extensions.label import LabelClasses
from pystac.extensions.label import LabelStatistics

from pystac.extensions.version import ItemVersionExtension

In [4]:
source_item = pystac.read_file(
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"
)

source_item

<Item id=S2A_10TFK_20220524_0_L2A>

In [5]:
for t in ["train", "test", "validate"]:
    label_item = pystac.Item(
        id=f"label_{t}",
        geometry=source_item.geometry,
        bbox=source_item.bbox,
        datetime=datetime.utcnow(),
        properties={"ml-aoi:split": t},
    )

    label = LabelExtension.ext(label_item, add_if_missing=True)
    label.apply(
        label_description="Land cover labels",
        label_type=LabelType.VECTOR,
        label_tasks=["segmentation", "regression"],
        label_classes=[LabelClasses({"water": 1, "urban": 2, "vegetation": 3})],
        label_methods=["manual"],
        label_properties=["class"],
    )
    label.add_source(source_item=source_item)
    label.add_geojson_labels(f"label-{t}.geojson")

    version = ItemVersionExtension(label_item)

    version.apply(version="0.1", deprecated=False)

    label_item.stac_extensions.append(
        "https://stac-extensions.github.io/version/v1.2.0/schema.json"
    )

    pystac.write_file(label_item, dest_href=f"item-label-{t}.json")

In [6]:
for t in ["train", "test", "validate"]:
    ml_aoi_item = pystac.Item(
        id=f"ml-aoi-{t}",
        geometry=source_item.geometry,
        bbox=source_item.bbox,
        datetime=datetime.utcnow(),
        properties={"ml-aoi:split": t},
    )
    ml_aoi_item.add_link(
        Link(
            rel="derived_from",
            target="https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A",
            extra_fields={"ml-aoi:role": "feature"},
        )
    )

    ml_aoi_item.add_link(
        Link(
            rel="derived_from",
            target=f"label-item-{t}.json",
            extra_fields={"role": "ml-aoi:label"},
        )
    )

    label_asset = Asset(
        href="labels-v1.geojson",
        description="label",
        media_type=pystac.MediaType.GEOJSON,
        roles=["label"],
        extra_fields={"ml-aoi:role": "label"},
    )

    ml_aoi_item.add_asset(key="label", asset=label_asset)

    ml_aoi_item.stac_extensions = [
        "https://stac-extensions.github.io/ml-aoi/v0.1.0/schema.json"
    ]

    pystac.write_file(ml_aoi_item, dest_href=f"item-ml-aoi-{t}.json")